### 1. 라이브러리 불러오기

In [1]:
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow import keras
%matplotlib inline

### 2. 데이터 불러오기 및 분리

In [3]:
traindir = "../images/mnist/trainingSet/"
testdir = "../images/mnist/testSet/"

In [4]:
IMG_SIZE = 32

In [5]:
def data_load (rootdir, in_img_size, ):
    imageList = []
    labelList = []
    IMG_SIZE = in_img_size
    categories = os.listdir(rootdir)
    # 폴더 순환
    for i in range (0, len(categories)):
        labelPath = os.path.join(rootdir,categories[i])
        imagePath = os.listdir(labelPath)
        # 이미지 순환
        for j in range(0, len(imagePath)):
            imageFullPath = os.path.join(labelPath, imagePath[j])
            try:
                baseImg = cv2.imread(imageFullPath, cv2.IMREAD_COLOR)
                ### 1. 이미지 사이즈 변경
                refinedImg = cv2.resize(baseImg, (IMG_SIZE,IMG_SIZE))
                imageList.append(refinedImg)
                labelList.append(i)
            except Exception as e:
                print(e, imageFullPath)
                pass

    imageArray = np.array(imageList)
    labelArray = np.array(labelList)
    
    
    return imageArray, labelArray

In [6]:
x_train, y_train = data_load(traindir, IMG_SIZE)
x_test, y_test = data_load(testdir, IMG_SIZE)

In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(42000, 32, 32, 3)
(42000,)
(200, 32, 32, 3)
(200,)


In [8]:
trainingData_features_norm = x_train/255.0
testData_features_norm = x_test/255.0

In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
trainingData_labels_one = to_categorical(y_train)
testData_labels_one = to_categorical(y_test)

In [11]:
trainingData_labels_one[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

### 3. 모델 생성

In [29]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

In [19]:
inputShape = trainingData_features_norm[0].shape
inputShape

(32, 32, 3)

In [20]:
outputShape = len(np.unique(y_train))
outputShape

10

In [21]:
from tensorflow.keras.applications.vgg16 import VGG16

In [22]:
vgg_model = VGG16(weights=None , include_top=False)

In [31]:
# 커스텀 input_shape 생성
keras_input = Input(shape=inputShape, name = 'custom_input')
keras_input.shape
# 커스텀모델 생성 
vgg_model_c = vgg_model(inputs=keras_input)

# Fully Connected Layer 추가 생성 
x = Flatten(name='flatten')(vgg_model_c)
x = Dense(64, activation="relu", name='custom_1')(x)
x = Dense(64, activation="relu", name='custom_2')(x)
x = Dense(outputShape, activation='softmax', name='prediction')(x)

# 커스텀 모델 생성
pretrained_model = Model(inputs=keras_input, outputs=x)
pretrained_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
custom_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
custom_1 (Dense)             (None, 64)                32832     
_________________________________________________________________
custom_2 (Dense)             (None, 64)                4160      
_________________________________________________________________
prediction (Dense)           (None, 10)                650       
Total params: 14,752,330
Trainable params: 14,752,330
Non-trainable params: 0
_________________________________________________

### 4. 모델 컴파일

In [32]:
pretrained_model.compile(loss="categorical_crossentropy", 
              optimizer= "adam",
              metrics=["accuracy"] )

### 5. 모델 훈련

In [33]:
pretrained_model.fit(x=trainingData_features_norm,
                     y=trainingData_labels_one,
                    epochs=3,
                    validation_data = (testData_features_norm, testData_labels_one))

Train on 42000 samples, validate on 200 samples
Epoch 1/3
 4160/42000 [=>............................] - ETA: 23:27 - loss: 2.3031 - accuracy: 0.0993

KeyboardInterrupt: 